In [ ]:
from revChatGPT.V1 import Chatbot
import json
f = open('config.json')
config = json.load(f)

chatbot = Chatbot(config={
  "access_token": config['access_token_lab']
}) 

In [ ]:
def GPT4_debug(chatbot, prompt, model="text-davinci-002-render-paid"):
    for data in chatbot.ask(prompt, conversation_id=None, parent_id=None, model=model):
        response = data["message"]
    print(response)
    # GPT4_clear_conversations(chatbot)
    return response


In [ ]:
GPT4_debug(chatbot, 'test')

In [ ]:
import pandas as pd
import time
import os
import sys

In [ ]:
tmp_config = {
    "dateset": "./data/",
    "output": "predicted_sql.txt"
}
# if sys.argv[1] == "--dataset" and sys.argv[3] == "--output":
DATASET_SCHEMA = tmp_config['dateset']+"tables.json"
DATASET = tmp_config['dateset']+"dev.json"
OUTPUT_FILE = tmp_config['output']
BEFORE_SQL = "sql_norm_del_value_record_get_before.txt"
# else:
#     raise Exception("Please use this format python CoT.py --dataset data/ --output predicted_sql.txt")


In [ ]:

def load_data(DATASET):
  return pd.read_json(DATASET)

def load_sql_txt(file_path):
  with open(file_path, 'r') as f:
    return [line.strip() for line in f]

def hard_prompt_maker(test_sample_text,database,schema_links,sub_questions):
  instruction = tmp2
  instruction += "# Use the intermediate representation and the schema links to generate the SQL queries for each of the questions.\n"
  fields = find_fields_MYSQL_like("college_2")
  fields += "Foreign_keys = " + find_foreign_keys_MYSQL_like("college_2") + '\n'
  fields += find_fields_MYSQL_like(database)
  fields += "Foreign_keys = " + find_foreign_keys_MYSQL_like(database) + '\n'
  stepping = f'''\nA: Let's think step by step. "{test_sample_text}" can be solved by knowing the answer to the following sub-question "{sub_questions}".'''
  fields += "\n"
  prompt = instruction +fields + hard_prompt + 'Q: "' + test_sample_text + '"' + '\nschema_links: ' + schema_links + stepping +'\nThe SQL query for the sub-question"'
  return prompt
def medium_prompt_maker(test_sample_text,database,schema_links):
  instruction = tmp2
  instruction += "# Use the the schema links and Intermediate_representation to generate the SQL queries for each of the questions.\n"
  fields = find_fields_MYSQL_like("college_2")
  fields += "Foreign_keys = " + find_foreign_keys_MYSQL_like("college_2") + '\n'
  fields += find_fields_MYSQL_like(database)
  fields += "Foreign_keys = " + find_foreign_keys_MYSQL_like(database) + '\n'
  fields += "\n"
  prompt = instruction +fields + medium_prompt + 'Q: "' + test_sample_text + '\nSchema_links: ' + schema_links + '\nA: Let’s think step by step.'
  return prompt
def easy_prompt_maker(test_sample_text,database,schema_links):
  instruction = "# Use the the schema links to generate the SQL queries for each of the questions.\n"
  fields = find_fields_MYSQL_like("college_2")
  fields += find_fields_MYSQL_like(database)
  fields += "\n"
  prompt = instruction +fields + easy_prompt + 'Q: "' + test_sample_text + '\nSchema_links: ' + schema_links + '\nSQL:'
  return prompt
def classification_prompt_maker(test_sample_text,database,schema_links):
  instruction = "# For the given question, classify it as EASY, NON-NESTED, or NESTED based on nested queries and JOIN.\n"
  instruction += "\nif need nested queries: predict NESTED\n"
  instruction += "elif need JOIN and don't need nested queries: predict NON-NESTED\n"
  instruction += "elif don't need JOIN and don't need nested queries: predict EASY\n\n"
  fields = find_fields_MYSQL_like("college_2")
  fields += "Foreign_keys = " + find_foreign_keys_MYSQL_like("college_2") + '\n'
  fields += find_fields_MYSQL_like(database)
  fields += "Foreign_keys = " + find_foreign_keys_MYSQL_like(database) + '\n'
  fields += "\n"
  prompt = instruction + fields + classification_prompt + 'Q: "' + test_sample_text + '\nschema_links: ' + schema_links + '\nA: Let’s think step by step.'
  return prompt
def schema_linking_prompt_maker(test_sample_text,database):
  instruction = "# Find the schema_links for generating SQL queries for each question based on the database schema and Foreign keys.\n"
  fields = find_fields_MYSQL_like(database)
  foreign_keys = "Foreign_keys = " + find_foreign_keys_MYSQL_like(database) + '\n'
  prompt = instruction + schema_linking_prompt + fields +foreign_keys+ 'Q: "' + test_sample_text + """"\nA: Let’s think step by step."""
  return prompt
def find_foreign_keys_MYSQL_like(db_name):
  df = spider_foreign[spider_foreign['Database name'] == db_name]
  output = "["
  for index, row in df.iterrows():
    output += row['First Table Name'] + '.' + row['First Table Foreign Key'] + " = " + row['Second Table Name'] + '.' + row['Second Table Foreign Key'] + ','
  output= output[:-1] + "]"
  return output
def find_fields_MYSQL_like(db_name):
  df = spider_schema[spider_schema['Database name'] == db_name]
  df = df.groupby(' Table Name')
  output = ""
  for name, group in df:
    output += "Table " +name+ ', columns = ['
    for index, row in group.iterrows():
      output += row[" Field Name"]+','
    output = output[:-1]
    output += "]\n"
  return output
def find_primary_keys_MYSQL_like(db_name):
  df = spider_primary[spider_primary['Database name'] == db_name]
  output = "["
  for index, row in df.iterrows():
    output += row['Table Name'] + '.' + row['Primary Key'] +','
  output = output[:-1]
  output += "]\n"
  return output
def creatiing_schema(DATASET_JSON):
    schema_df = pd.read_json(DATASET_JSON)
    schema_df = schema_df.drop(['column_names','table_names'], axis=1)
    schema = []
    f_keys = []
    p_keys = []
    for index, row in schema_df.iterrows():
        tables = row['table_names_original']
        col_names = row['column_names_original']
        col_types = row['column_types']
        foreign_keys = row['foreign_keys']
        primary_keys = row['primary_keys']
        for col, col_type in zip(col_names, col_types):
            index, col_name = col
            if index == -1:
                for table in tables:
                    schema.append([row['db_id'], table, '*', 'text'])
            else:
                schema.append([row['db_id'], tables[index], col_name, col_type])
        for primary_key in primary_keys:
            index, column = col_names[primary_key]
            p_keys.append([row['db_id'], tables[index], column])
        for foreign_key in foreign_keys:
            first, second = foreign_key
            first_index, first_column = col_names[first]
            second_index, second_column = col_names[second]
            f_keys.append([row['db_id'], tables[first_index], tables[second_index], first_column, second_column])
    spider_schema = pd.DataFrame(schema, columns=['Database name', ' Table Name', ' Field Name', ' Type'])
    spider_primary = pd.DataFrame(p_keys, columns=['Database name', 'Table Name', 'Primary Key'])
    spider_foreign = pd.DataFrame(f_keys,
                        columns=['Database name', 'First Table Name', 'Second Table Name', 'First Table Foreign Key',
                                 'Second Table Foreign Key'])
    return spider_schema,spider_primary,spider_foreign
def debuger(test_sample_text,database,sql):
  instruction = """#### For the given question, use the provided tables, columns, foreign keys, and primary keys to fix the given SQLite SQL QUERY for any issues. If there are any problems, fix them. If there are no issues, return the SQLite SQL QUERY as is.
#### Use the following instructions for fixing the SQL QUERY:
1) Use the database values that are explicitly mentioned in the question.
2) Pay attention to the columns that are used for the JOIN by using the Foreign_keys.
3) Use DESC and DISTINCT when needed.
4) Pay attention to the columns that are used for the GROUP BY statement when using SUM, AVG, MAX, MIN and COUNT.
5) Pay attention to the columns that are used for the SELECT statement.
6) Only change the GROUP BY clause when necessary (Avoid redundant columns in GROUP BY).
7) Use GROUP BY on one column only.

"""
  fields = find_fields_MYSQL_like(database)
  fields += "Foreign_keys = " + find_foreign_keys_MYSQL_like(database) + '\n'
  fields += "Primary_keys = " + find_primary_keys_MYSQL_like(database)
  prompt = instruction + fields+ '#### Question: ' + test_sample_text + '\n#### SQLite SQL QUERY\n' + sql +'\n#### SQLite FIXED SQL QUERY\nSELECT'
  return prompt
def GPT4_generation(prompt):
  for data in chatbot.ask(prompt, conversation_id=config['conversation_id'], parent_id=None):
    response = data["message"]
  GPT4_clear_conversations(chatbot)
  return response

def GPT4_debug(prompt):
  for data in chatbot.ask(prompt, conversation_id=config['conversation_id'], parent_id=None):
    response = data["message"]
  GPT4_clear_conversations(chatbot)
  return response

def GPT4_clear_conversations(chatbot):
  chatbot.clear_conversations()


In [ ]:

spider_schema,spider_primary,spider_foreign = creatiing_schema(DATASET_SCHEMA)
val_df = load_data(DATASET)
SQL_list = load_sql_txt(BEFORE_SQL)
crt_time = time.strftime("%m-%d-%H:%M:%S", time.localtime())
print(f"Number of data samples {val_df.shape[0]}")
start_index = 451
end_index = 524
with open(f'fix_debug-{start_index}-{end_index}-{crt_time}.log', 'w') as record, open(f'fix_debug_record-{start_index}-{end_index}-{crt_time}.log', 'w') as schema_link:
    CODEX = []
    for ((index, row), SQL) in zip(val_df.iterrows(), SQL_list):
        if index < start_index: continue #for testing
        print("index:", index)
        # if index < 10: continue #for testing
        record.write(f"\nindex is {index}"+ '\n')
        record.write(row['query']+ '\n')
        record.write(row['question']+ '\n')
        
        record.write('SQL generation:'+ '\n')
        record.write(SQL+ '\n')
        debugged_SQL = None
        while debugged_SQL is None:
            try:
                with open('tmp.txt', 'w') as f:
                    f.write(debuger(row['question'], row['db_id'], SQL))
                debugged_SQL = GPT4_debug(debuger(row['question'], row['db_id'], SQL)).replace("\n", " ")
            except:
                time.sleep(3)
                pass
        SQL = "SELECT " + debugged_SQL
        record.write('self correction:'+ '\n')
        record.write(SQL+ '\n')
        schema_link.write(SQL + '\n')
        CODEX.append([row['question'], SQL, row['query'], row['db_id']])
        #break
        # if index == 30: break
        if index == end_index: break
# df = pd.DataFrame(CODEX, columns=['NLQ', 'PREDICTED SQL', 'GOLD SQL', 'DATABASE'])
# results = df['PREDICTED SQL'].tolist()
# with open(OUTPUT_FILE, 'w') as f:
#     for line in results:
#         f.write(f"{line}\n")

In [1]:
record_file_name = 'my_result/origin/2/new_classification/code_tag_classification_record'
with open(f'{record_file_name}.log', 'r') as record:
    record_line = [line.strip() for line in record]
    
# cmp_result_file_name = '/home/r10525068/ensemble/result/gpt-3.5/din_sql/origin/2/din_sql_before_cmp_result.txt'
# with open(f'{cmp_result_file_name}', 'r') as record:
#     result_line = [line.strip() for line in record]

In [ ]:
with open(f'{record_file_name}_cmp_result_before.txt', 'w') as cmp_result:
    for index, line in enumerate(result_line):
        if 'Correct:' in line:
            cmp_result.write(line+'\n')
            # cmp_result.write(line.split('Correct: ')[1]+'\n')

In [ ]:
with open(f'{record_file_name}_cmp_result_before.txt', 'r') as cmp_result:
    cmp_result_list = [line.strip() for line in cmp_result]

In [ ]:
len(cmp_result_list)

In [ ]:
# schema_links
with open(f'{record_file_name}_schema_link.txt', 'w') as schema_link:
    crt_index = -1
    for index, line in enumerate(record_line):
        # if 'index is' in line:
        #     crt_index = int(record_line[index].split('index is ')[1])
        #     schema_link.write(f'index: {crt_index}\n')
        #     schema_link.write(f'Gold SQL: {record_line[index+1]}\n')
        if 'schema_links:' in line:
            # schema_link.write(f'schema_links: {record_line[index+1]}\n')
            schema_link.write(f'{record_line[index+1]}\n')
            # schema_link.write(f'{cmp_result_list[crt_index]}\n\n')


In [ ]:
# SQL生成到自我修正
with open(f'{record_file_name}_get_before_after.txt', 'w') as schema_link,  open(f'{record_file_name}_get_before.txt', 'w') as before,  open(f'{record_file_name}_get_after.txt', 'w') as after:
    for index, line in enumerate(record_line):
        if 'index is' in line:
            schema_link.write(record_line[index] + '\n')
        if 'self correction:' in line:
            schema_link.write('before:\n')
            schema_link.write(record_line[index-1] + '\n')
            before.write(record_line[index-1] + '\n')
            schema_link.write('after:\n')
            schema_link.write(record_line[index+1] + '\n\n')
            after.write(record_line[index+1] + '\n')


In [2]:
# 分類結果
nested_str = ['INTERSECT', 'UNION', 'EXCEPT', 'IN', 'NOT IN']
with open(f'{record_file_name}_check_classifiaction.txt', 'w') as f:
    crt_index = -1
    results = []
    result = {}
    for index, line in enumerate(record_line):
        if 'index is' in line:
            if result:
                result['pred_label'] = ''
                f.write(f'Pred Label: {result["pred_label"]}\n')
                result['correct'] = (result['gold_label'] == result['pred_label'])
                f.write(f'Correct: {result["correct"]}\n\n')
                results.append(result)
                result = {}
            crt_index = record_line[index]
            result['index'] = crt_index
            f.write(record_line[index] + '\n')
            gold_sql = record_line[index+1]
            f.write(f'Gold SQL: {gold_sql}\n')
            result['gold'] = gold_sql
            gold_sql_list = gold_sql.lower().split(' ')
            if gold_sql_list.count('select') > 1 or gold_sql_list.count('(select'):
                result['gold_label'] = 'nested'
            elif 'join' in gold_sql.lower():
                result['gold_label'] = 'non-nested'
            else:
                result['gold_label'] = 'easy'
            f.write(f'Gold Label: {result["gold_label"]}\n')
        if 'Label:' in line and result:
            try:
                result['pred_label'] = line.split('"')[1].lower().strip()
            except:
                result['pred_label'] = line.split('Label:')[1].lower().strip()
            f.write(f'Pred Label: {result["pred_label"]}\n')
            result['correct'] = (result['gold_label'] == result['pred_label'])
            f.write(f'Correct: {result["correct"]}\n\n')
            results.append(result)
            result = {}


In [2]:
# 只分成 有無 JOIN 的分類結果
with open(f'{record_file_name}_check_two_classifiaction.txt', 'w') as f:
    crt_index = -1
    results = []
    result = {}
    for index, line in enumerate(record_line):
        if 'index is' in line:
            if result:
                result['pred_label'] = ''
                f.write(f'Pred Label: {result["pred_label"]}\n')
                result['correct'] = (result['gold_label'] == result['pred_label'])
                f.write(f'Correct: {result["correct"]}\n\n')
                results.append(result)
                result = {}
            crt_index = record_line[index]
            result['index'] = crt_index
            f.write(record_line[index] + '\n')
            gold_sql = record_line[index+1]
            f.write(f'Gold SQL: {gold_sql}\n')
            result['gold'] = gold_sql
            gold_sql_list = gold_sql.lower().split(' ')
            if gold_sql_list.count('join') or gold_sql_list.count('(select'):
                result['gold_label'] = 'complex'
            else:
                result['gold_label'] = 'easy'
            f.write(f'Gold Label: {result["gold_label"]}\n')
        if 'Label:' in line and result:
            try:
                result['pred_label'] = line.split('"')[1].lower().strip()
            except:
                result['pred_label'] = line.split('Label:')[1].lower().strip()
            f.write(f'Pred Label: {result["pred_label"]}\n')
            result['correct'] = (result['gold_label'] == result['pred_label'])
            f.write(f'Correct: {result["correct"]}\n\n')
            results.append(result)
            result = {}


In [3]:
len(results)

1034

In [4]:
label = {'easy': 0, 'non-nested': 1, 'nested': 2, '': 3}
# gold_to_pred
analysis = [
    {'label': 'easy_to_easy', 'record': []},
    {'label': 'easy_to_non', 'record': []},
    {'label': 'easy_to_nested', 'record': []},
    {'label': 'easy_to_', 'record': []},
    {'label': 'non_to_easy', 'record': []},
    {'label': 'non_to_non', 'record': []},
    {'label': 'non_to_nested', 'record': []},
    {'label': 'non_to_', 'record': []},
    {'label': 'nested_to_easy', 'record': []},
    {'label': 'nested_to_non', 'record': []},
    {'label': 'nested_to_nested', 'record': []},
    {'label': 'nested_to_', 'record': []},
]

for i, result in enumerate(results):
    # if i < 1020: continue
    try:
        idx = label[result['gold_label']]*4 + label[result['pred_label']]
    except:
        idx = label[result['gold_label']]*4 + label['']
    # print(result['gold_label'], result['pred_label'])
    # print(idx)
    # print()
    analysis[idx]['record'].append(result)
    # if i > 349: break

# with open(f'{record_file_name}_get_classification_analysis.txt', 'w') as f:
for ana in analysis:
    print(ana['label'] + ': ' + str(len(ana['record'])))
        # f.write(ana['label'] + ': ' + str(len(ana['record'])) + '\n')

easy_to_easy: 426
easy_to_non: 50
easy_to_nested: 60
easy_to_: 8
non_to_easy: 74
non_to_non: 188
non_to_nested: 63
non_to_: 6
nested_to_easy: 18
nested_to_non: 31
nested_to_nested: 110
nested_to_: 0


In [5]:
with open(f'{record_file_name}_three_classification.txt', 'w') as f:
    for result in results:
        f.write(result['pred_label']+'\n')

In [7]:
record_file_name

'my_result/origin/2/new_classification/short_question_classification_record'